## SetUP

In [49]:
%pip install --upgrade pip
%pip install LangChain --quiet
%pip install langchain-community --quiet
%pip install faiss-cpu --quiet
%pip install -U langchain-aws --quiet
%pip install amazon-textract-caller --quiet
%pip install amazon-textract-textractor --quiet
%pip install anthropic --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [47]:
import pandas as pd
import numpy as np
import re
from tqdm.auto import tqdm
import boto3

from langchain_community.document_loaders import AmazonTextractPDFLoader
from langchain_aws import ChatBedrock
from langchain.llms import Bedrock
from langchain.chains import LLMChain
from langchain_community.embeddings import BedrockEmbeddings
from langchain.chains.summarize import load_summarize_chain

from langchain_community.vectorstores import FAISS
from langchain_core.example_selectors import (
    MaxMarginalRelevanceExampleSelector,
    SemanticSimilarityExampleSelector,
)
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

## Prompt engineering (Classification)

In [7]:
# examples with the abstract of the papers
examples = [
    {
    "Text": "The correct use of model evaluation, model selection, and algorithm selection techniques is vital in academic machine learning research as well as in many industrial settings. This article reviews different techniques that can be used for each of these three subtasks and discusses the main advantages and disadvantages of each technique with references to theoretical and empirical studies. Further, recommendations are given to encourage best yet feasible practices in research and applications of machine learning. Common methods such as the holdout method for model evaluation and selection are covered, which are not recommended when working with small datasets. Different flavors of the bootstrap technique are introduced for estimating the uncertainty of performance estimates, as an alternative to confidence intervals via normal approximation if bootstrapping is computationally feasible. Common cross-validation techniques such as leave-oneout cross-validation and k-fold cross-validation are reviewed, the bias-variance trade-off for choosing k is discussed, and practical tips for the optimal choice of k are given based on empirical evidence. Different statistical tests for algorithm comparisons are presented, and strategies for dealing with multiple comparisons such as omnibus tests and multiple-comparison corrections are discussed. Finally, alternative methods for algorithm selection, such as the combined F-test 5x2 crossvalidation and nested cross-validation, are recommended for comparing machine learning algorithms when datasets are small.",
    "Output": "General ML",
    },
    {
    "Text": "As one of the most successful approaches to building recommender systems,collaborative filtering(CF) uses the known preferences of a group of users to make recommendations or predictions of the unknown preferences for other users. In this paper, we first introduce CF tasks and their main challenges, such as data sparsity, scalability, synonymy, gray sheep, shilling attacks, privacy protection, etc., and their possible solutions. We then present three main categories of CF techniques: memory-based, modelbased, and hybrid CF algorithms (that combine CF with other recommendation techniques), with examples for representative algorithms of each category, and analysis of their predictive performance and their ability to address the challenges. From basic techniques to the state-of-the-art, we attempt to present a comprehensive survey for CF techniques, which can be served as a roadmap for research and practice in this area.",
    "Output": "Recommenders", 
    },
    { 
    "Text": "This paper is an attempt to explain all the matrix calculus you need in order to understand the training of deep neural networks. We assume no math knowledge beyond what you learned in calculus 1, and provide links to help you refresh the necessary math where needed. Note that you do not need to understand this material before you start learning to train and use deep learning in practice; rather, this material is for those who are already familiar with the basics of neural networks, and wish to deepen their understanding of the underlying math. Don’t worry if you get stuck at some point along the way—just go back and reread the previous section, and try writing down and working through some examples. And if you’re still stuck, we’re happy to answer your questions in the Theory category at forums.fast.ai. Note: There is a reference section at the end of the paper summarizing all the key matrix calculus rules and terminology discussed here.",
    "Output": "Neural Networks",
    },
    {
    "Text": "Deeper neural networks are more difficult to train. We present a residual learning framework to ease the training of networks that are substantially deeper than those used previously. We explicitly reformulate the layers as learning residual functions with reference to the layer inputs, instead of learning unreferenced functions. We provide comprehensive empirical evidence showing that these residual networks are easier to optimize, and can gain accuracy from considerably increased depth. On the ImageNet dataset we evaluate residual nets with a depth of up to 152 layers—8× deeper than VGG nets [41] but still having lower complexity. An ensemble of these residual nets achieves 3.57% error onthe ImageNet test set. This result won the 1st place on the ILSVRC 2015 classification task. We also present analysis on CIFAR-10 with 100 and 1000 layers. training error (%) 10 Jian Sun 20 56-layer 20-layer 0 test error (%) 10 0 56-layer 20-layer 0 6 5 4 3 2 1 0 iter. (1e4) 1 2 3 4 iter. (1e4) 5 6 Figure 1. Training error (left) and test error (right) on CIFAR-10 with 20-layer and 56-layer “plain” networks. The deeper network has higher training error, and thus test error. Similar phenomena on ImageNet is presented in Fig. 4. greatly benefited from very deep models. The depth of representations is of central importance for many visual recognition tasks. Solely due to our extremely deep representations, we obtain a 28% relative improvement on the COCO object detection dataset. Deep residual nets are foundations of our submissions to ILSVRC & COCO 2015 competitions1, where we also won the 1st places on the tasks of ImageNet detection, ImageNet localization, COCO detection, and COCO segmentation.",
    "Output": "Object Detection", 
    },
    {
    "Text": "Over the past few years, neural networks have re-emerged as powerful machine-learning models, yielding state-of-the-art results in fields such as image recognition and speech processing. More recently, neural network models started to be applied also to textual natural language signals, again with very promising results. This tutorial surveys neural network models from the perspective of natural language processing research, in an attempt to bring natural-language researchers up to speed with the neural techniques. The tutorial covers input encoding for natural language tasks, feed-forward networks, convolutional networks, recurrent networks and recursive networks, as well as the computation graph abstraction for automatic gradient computation",
    "Output": "NLP",
    },  
]

In [54]:
prompt_claude = """

<instructions>
You are a Scientific paper system. 
Please do the following:
1.  Given a list of classes, 
    classify the document into one of the classes into the <classes></classes> tags.
    Think your answer with the following reasoning: 
    First, check the examples showed into the <example></example> tags. 
    Second, list CLUES like: 
    What is the paper about?,
    What kind of issue the paper is addressing?, 
    What kind of machine learning field this paper is related? and why?.
    Before reply add your reasoning into the <thinking></thinking> tags.
    Skip any preamble text and provide your final answer 
    just replay with the class name within <label></label> tags.
</instructions>

<classes>General ML, Recommenders, Neural Networks, Object Detection, NLP</classes>

here are some examples of text documents with their expected output: <example>""" 

suffix_template="""
</example>
<document>{doc_text}</document>
<label></label>
"""

In [55]:
example_prompt = PromptTemplate(
    input_variables=["Text", "Output"],
    template="Text: {Text}\nOutput: {Output}",
)

In [56]:
example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    # The list of examples available to select from.
    examples,
    # The embedding class used to produce embeddings which are used to measure semantic similarity.
    BedrockEmbeddings(model_id= 'amazon.titan-embed-text-v1'),
    # The VectorStore class that is used to store the embeddings and do a similarity search over.
    FAISS,
    # The number of examples to produce.
    k=1,
)

In [57]:
text_examp = "The Machine Learning field evolved from the broad field of Artificial Intelligence, which aims to mimic intelligent abilities of humans by machines. In the field of Machine Learning one considers the important question of how to make machines able to “learn”. Learning in this context is understood as inductive inference, where one observes examples that represent incomplete information about some “statistical phenomenon”. In unsupervised learning one typically tries to uncover hidden regularities (e.g. clusters) or to detect anomalies in the data (for instance some unusual machine function or a network intrusion). In supervised learning, there is a label associated with each example. It is supposed to be the answer to a question about the example. If the label is discrete, then the task is called classification problem– otherwise, for realvalued labels we speak of a regression problem. Based on these examples (including the labels), one is particularly interested to predict the answer for other cases before they are explicitly observed. Hence, learning is not only a question of remembering but also of generalization to unseen cases"
selected_examples = example_selector.select_examples({"Text": text_examp})
for example in selected_examples:
    print("\n")
    for k, v in example.items():
        print(f"{k}: {v}")



Text: The correct use of model evaluation, model selection, and algorithm selection techniques is vital in academic machine learning research as well as in many industrial settings. This article reviews different techniques that can be used for each of these three subtasks and discusses the main advantages and disadvantages of each technique with references to theoretical and empirical studies. Further, recommendations are given to encourage best yet feasible practices in research and applications of machine learning. Common methods such as the holdout method for model evaluation and selection are covered, which are not recommended when working with small datasets. Different flavors of the bootstrap technique are introduced for estimating the uncertainty of performance estimates, as an alternative to confidence intervals via normal approximation if bootstrapping is computationally feasible. Common cross-validation techniques such as leave-oneout cross-validation and k-fold cross-vali

In [58]:
mmr_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=prompt_claude, #A prompt template string to put before the examples.
    suffix=suffix_template, #A prompt template string to put after the examples.
    input_variables=["doc_text"],
)

In [59]:
print(mmr_prompt.format(doc_text=text_examp))



<instructions>
You are a Scientific paper system. 
Please do the following:
1.  Given a list of classes, 
    classify the document into one of the classes into the <classes></classes> tags.
    Think your answer with the following reasoning: 
    First, check the examples showed into the <example></example> tags. 
    Second, list CLUES like: 
    What is the paper about?,
    What kind of issue the paper is addressing?, 
    What kind of machine learning field this paper is related? and why?.
    Before reply add your reasoning into the <thinking></thinking> tags.
    Skip any preamble text and provide your final answer 
    just replay with the class name within <label></label> tags.
</instructions>

<classes>General ML, Recommenders, Neural Networks, Object Detection, NLP</classes>

here are some examples of text documents with their expected output: <example>

Text: The correct use of model evaluation, model selection, and algorithm selection techniques is vital in academic mach

In [36]:
# load paper from s3
# Initialize S3 client and set RoleArn, region name, and bucket name
s3 = boto3.client("s3")
roleArn = 'arn:aws:iam::013987100154:role/textractTest'
region_name = 'us-east-1'
bucket_name = 'llm-showcase'

# to hold all docs in bucket
docs_list = []

# loop through docs in bucket, get names of all docs
s3_resource = boto3.resource("s3")
bucket = s3_resource.Bucket(bucket_name)
for bucket_object in bucket.objects.filter(Prefix='papers/', Delimiter='/'):
    if bucket_object.key.endswith('.pdf'):
        docs_list.append(bucket_object.key)

        
file_path = f"s3://llm-showcase/" + docs_list[0]
loader = AmazonTextractPDFLoader(file_path)
document = loader.load()

In [39]:
print("cantidad de paginas {}".format(len(document)))

cantidad de paginas 6


### document summarization

In [50]:
#summarize paper with map reduce component
bedrock = boto3.client(service_name='bedrock-runtime')
bedrock_llm = BedrockChat(client=bedrock, model_id="anthropic.claude-3-sonnet-20240229-v1:0")

summary_chain = load_summarize_chain(llm=bedrock_llm, chain_type='map_reduce')
output = summary_chain.invoke(document)

In [51]:
output

{'input_documents': [Document(page_content='A Brief Introduction into Machine Learning\n\n\nGunnar Rätsch\n\n\nFriedrich Miescher Laboratory of the Max Planck Society,\n\n\nSpemannstraße 37, 72076 Tübingen, Germany\n\n\nttp://www.tuebingen.mpg.de/-raetsch\n\n\n1 Introduction\n\n\nThe Machine Learning field evolved from the broad field of Artificial Intelligence,\n\n\nwhich aims to mimic intelligent abilities of humans by machines. In the field of Ma-\n\n\nchine Learning one considers the important question of how to make machines able\n\n\nto "learn". Learning in this context is understood as inductive inference, where one\n\n\nobserves examples that represent incomplete information about some "statistical phe-\n\n\nnomenon". In unsupervised learning one typically tries to uncover hidden regularities\n\n\n(e.g. clusters) or to detect anomalies in the data (for instance some unusual machine\n\n\nfunction or a network intrusion). In supervised learning, there is a label associated\n\n\nw

In [52]:
output['output_text']

'The given text provides an introduction to machine learning, focusing on supervised classification tasks. It defines machine learning as learning from examples to uncover patterns or make predictions on unseen data. The text discusses common classification algorithms like k-Nearest Neighbors, Linear Discriminant Analysis, Decision Trees, Naive Bayes, Support Vector Machines (SVMs), and Boosting. It highlights the importance of feature selection and pattern representation. The text then delves into the theoretical foundations of machine learning, including statistical learning theory and the Universal Approximation Theorem for neural networks. It emphasizes the widespread applicability of machine learning techniques across various domains, such as text classification, bioinformatics, and particle physics. The text concludes with a list of references related to machine learning, neural networks, and SVMs, spanning books, theses, and research papers from the early foundations to more rec

In [60]:
bedrock = boto3.client(service_name='bedrock-runtime')
bedrock_llm = BedrockChat(client=bedrock, model_id="anthropic.claude-3-sonnet-20240229-v1:0")

llm_chain = LLMChain(prompt=mmr_prompt, llm=bedrock_llm)

llm_response = llm_chain.invoke(output['output_text'])
llm_response

{'doc_text': 'The given text provides an introduction to machine learning, focusing on supervised classification tasks. It defines machine learning as learning from examples to uncover patterns or make predictions on unseen data. The text discusses common classification algorithms like k-Nearest Neighbors, Linear Discriminant Analysis, Decision Trees, Naive Bayes, Support Vector Machines (SVMs), and Boosting. It highlights the importance of feature selection and pattern representation. The text then delves into the theoretical foundations of machine learning, including statistical learning theory and the Universal Approximation Theorem for neural networks. It emphasizes the widespread applicability of machine learning techniques across various domains, such as text classification, bioinformatics, and particle physics. The text concludes with a list of references related to machine learning, neural networks, and SVMs, spanning books, theses, and research papers from the early foundation

## Prompt engineering (Extract Key phrases)

In [63]:
prompt_keypoints = """ 

<instructions>
You are a Scientific paper system. 
Please do the following:
1. Extract the authors from the paper and add them into <author></author> tags.
2. Extract the paper's publish date and add it into <date></date> tags.
3. Extract the paper's title and add it into <title></title> tags.
</instructions>

<document>{doc_text}</document>

<author></author>
<date></date>
<title></title>
"""

In [65]:
prompt_key = PromptTemplate(template=prompt_keypoints, input_variables=["doc_text"])
bedrock_llm = BedrockChat(client=bedrock, model_id="anthropic.claude-3-sonnet-20240229-v1:0")
llm_chain = LLMChain(prompt=prompt_key, llm=bedrock_llm)
llm_response = llm_chain.invoke(document[0].page_content)
llm_response

{'doc_text': 'A Brief Introduction into Machine Learning\n\n\nGunnar Rätsch\n\n\nFriedrich Miescher Laboratory of the Max Planck Society,\n\n\nSpemannstraße 37, 72076 Tübingen, Germany\n\n\nttp://www.tuebingen.mpg.de/-raetsch\n\n\n1 Introduction\n\n\nThe Machine Learning field evolved from the broad field of Artificial Intelligence,\n\n\nwhich aims to mimic intelligent abilities of humans by machines. In the field of Ma-\n\n\nchine Learning one considers the important question of how to make machines able\n\n\nto "learn". Learning in this context is understood as inductive inference, where one\n\n\nobserves examples that represent incomplete information about some "statistical phe-\n\n\nnomenon". In unsupervised learning one typically tries to uncover hidden regularities\n\n\n(e.g. clusters) or to detect anomalies in the data (for instance some unusual machine\n\n\nfunction or a network intrusion). In supervised learning, there is a label associated\n\n\nwith each example. It is suppos